# Lesson 3 - Wrapping the QA Agent into an A2A Server

In this lesson, you will take the `PolicyAgent` created in Lesson 2 and wrap it into an Agent2Agent (A2A) server. This makes the agent discoverable and callable by other agents using the A2A protocol. You will define the agent's identity using an `AgentCard`, describe its capabilities using `AgentSkill`, and set up the request handling logic.


## 3.1. Define the A2A Server

You will write the server code to a file named `a2a_policy_agent.py`. This script performs the following steps:
1.  **Imports**: Loads necessary modules including the A2A SDK (`a2a.server`).
2.  **Executor**: Defines `PolicyAgentExecutor`, which bridges the A2A request context to your `PolicyAgent` class.
3.  **Metadata**: configure the `AgentSkill` (defining *what* it can do) and the `AgentCard` (defining *who* it is, including its URL).
4.  **Application**: Creates and runs an `A2AStarletteApplication` using `uvicorn`.


In [24]:
%%writefile a2a_policy_agent.py
from dotenv import load_dotenv
import os
import uvicorn

from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.apps import A2AStarletteApplication
from a2a.server.events import EventQueue
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
)
from a2a.utils import new_agent_text_message

from agents import PolicyAgent


class PolicyAgentExecutor(AgentExecutor):
    def __init__(self) -> None:
        self.agent = PolicyAgent()

    async def execute(
        self,
        context: RequestContext,
        event_queue: EventQueue,
    ) -> None:
        prompt = context.get_user_input()
        response = self.agent.answer_query(prompt)
        message = new_agent_text_message(response)
        await event_queue.enqueue_event(message)

    async def cancel(
        self,
        context: RequestContext,
        event_queue: EventQueue,
    ) -> None:
        pass

def main() -> None:
    print(f"Running A2A Health Insurance Policy Agent")
    _ = load_dotenv()
    PORT = int(os.environ.get("POLICY_AGENT_PORT", 9999))
    HOST = os.environ.get("AGENT_HOST", "localhost")

    skill = AgentSkill(
        id="insurance_coverage",
        name="Insurance coverage",
        description="Provides information about insurance coverage options and details.",
        tags=["insurance", "coverage"],
        examples=["What does my policy cover?", "Are mental health services included?"],
    )

    agent_card = AgentCard(
        name="InsurancePolicyCoverageAgent",
        description="Provides information about insurance policy coverage options and details.",
        url=f"http://{HOST}:{PORT}/",
        version="1.0.0",
        default_input_modes=["text"],
        default_output_modes=["text"],
        capabilities=AgentCapabilities(streaming=False),
        skills=[skill],
    )

    request_handler = DefaultRequestHandler(
        agent_executor=PolicyAgentExecutor(),
        task_store=InMemoryTaskStore(),
    )

    server = A2AStarletteApplication(
        agent_card=agent_card,
        http_handler=request_handler,
    )

    uvicorn.run(server.build(), host=HOST, port=PORT)

    
if __name__ == '__main__':
    main()
        

Overwriting a2a_policy_agent.py
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## 3.2. Run the Policy A2A Server

Now to activate your configured A2A agent, you would need to run your agent server. You can run the agent server using `uv`:

- Open the terminal by running the cell below
- Type `uv run a2a_policy_agent.py` to run the server and activate your A2A agent.

In [25]:
import os

from IPython.display import IFrame

url = os.environ.get("DLAI_LOCAL_URL").format(port=8888)
IFrame(f"{url}terminals/1", width=800, height=600)

## 3.3. Resources

- [Agent2Agent Protocol Documentation](https://a2a-protocol.org/)
- [A2A Python SDK Documentation](https://a2a-protocol.org/latest/sdk/python/api/)
- [Starlette Documentation](https://www.starlette.io/)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>
</div>
